In [2]:
from __future__ import print_function
import os
import pandas as pd
import math

from enum import Enum
import numpy as np

class Analysis(Enum):
    VERIFIED_RELIABLE = 'verified_reliable'
    GUESSED_RELIABLE = 'guessed_reliable'
    VERIFIED_CATEGORY = 'verified_category'

RELIABLE, UNRELIABLE = ["RELIABLE", "UNRELIABLE"]

class Load(object):
    data_path = "/opt/shared_venvs"
    
    @staticmethod
    def switch_y_for_npm(X, Y):
        for (i, (x, y)) in enumerate(zip(X, Y)):
            if "npm ERR" in x:
                Y[i] = UNRELIABLE

    def get_original_idx(self, idx_in_mangled_data):
        return self.log_path_to_idx[self.X_log_path[idx_in_mangled_data]]

    def get_mangled_idx(self, original_idx):
        return self.original_to_mangled[original_idx]

    def __init__(self, analysis=Analysis.VERIFIED_RELIABLE, min_num_occurence_of_category=6):
        categories = {}
        curr_idx = 0
        if analysis == Analysis.VERIFIED_RELIABLE:
            self.category_attribute = "verified_category"
            df = pd.read_csv(os.path.join(self.data_path, 'data', "verified",
                                          'with_content.csv'))
        elif analysis == Analysis.GUESSED_RELIABLE:
            self.category_attribute = "guessed_category"
            df = pd.read_csv(os.path.join(self.data_path, 'data', "unverified",
                                          'with_content.csv'))
        elif analysis == Analysis.VERIFIED_CATEGORY:
            self.category_attribute = "full_categories"
            df = pd.read_csv(os.path.join(self.data_path, 'data', "verified",
                                          'with_content.csv'))
            y = []
            for text, (verified, category) in zip(df['text'],
                                                  zip(df['verified_category'],
                                                      df['categories'])):
                new_cat = category
                if isinstance(new_cat, float) and math.isnan(new_cat):
                    if verified == 'RELIABLE':
                        new_cat = verified
                if new_cat not in categories:
                    curr_idx += 1
                    categories[new_cat] = (curr_idx, 0)
                idx, val = categories[new_cat]
                categories[new_cat] = (idx, val + 1)
                y.append(idx)
            new_cat = pd.Series(y, name=self.category_attribute)
            df = pd.concat([df, new_cat], axis=1)
        self.idx_per_category = {v[0]: k for k, v in categories.items()}
        # filter the categories that have at least min_num_occurence_of_category
        counts = df.groupby(self.category_attribute).aggregate(np.count_nonzero)
        chosen_categories = counts[counts.path_to_log >= min_num_occurence_of_category].index
        # print ["{}: {}".format(idx_per_category[k], categories[idx_per_category[k]][1]) for k in chosen_categories]
        self.log_path_to_idx = {path: idx for (idx, path) in enumerate(df['path_to_log'])}
        df = df[getattr(df, self.category_attribute).isin(chosen_categories)]
        self.X_log_path = np.array(list(df['path_to_log']))
        self.X_text = np.array(list(df['text']))
        self.mangled_to_original = {i: self.get_original_idx(i)
                                    for (i, _) in enumerate(self.X_text)}
        self.original_to_mangled = {v: k for k, v in self.mangled_to_original.items()}
        self.root_cause = []

        try:
            self.root_cause = [self.idx_per_category[x] for x in df['full_categories']]
        except Exception:
            pass
        Y = df[self.category_attribute]
        if analysis in (Analysis.VERIFIED_RELIABLE,):
            self.switch_y_for_npm(self.X_text, Y)
            unreliable = len([y for y in Y if y == "UNRELIABLE"])
            reliable = len([y for y in Y if y == "RELIABLE"])
            print("unreliable: {0}, reliable: {1}, ({2:.2f})".format(unreliable, reliable, unreliable / float(reliable + unreliable) * 100.0))

        self.Y = np.array(list(Y))


ImportError: Missing required dependencies ['numpy']

In [2]:
import findspark
# python_path must be the path of python in the workers
# i.e. if you installed libs on a virtualenv 
# available in all workers you should put
# findspark.init(python_path=<path_virtualenv>/bin/python)
findspark.init(python_path="/opt/sklearn_env/bin/python")
import os
from pyspark import SparkConf
from pyspark import SparkContext

conf = (SparkConf()
         .setMaster("spark://tlsisbld100l:7077")
         .setAppName("testPySparkIon")
         .set("spark.executor.memory", "1g")
         #.set("spark.cores.max", 100)
         .set("spark.broadcast.factory", "org.apache.spark.broadcast.HttpBroadcastFactory")
         .set("spark.driver.port", 7001)
         .set("spark.fileserver.port", 7002)
         .set("spark.broadcast.port", 7003)
         .set("spark.replClassServer.port", 7004)
         .set("spark.blockManager.port", 7005)
         .set("spark.executor.port", 7006))
sc = SparkContext(conf=conf)


In [3]:
"""
Class for parallelizing GridSearchCV jobs in scikit-learn
"""

from collections import Sized
import numpy as np

from sklearn.base import BaseEstimator, is_classifier, clone
from sklearn.cross_validation import KFold, check_cv, _fit_and_score, _safe_split
from sklearn.grid_search import BaseSearchCV, _check_param_grid, ParameterGrid, _CVScoreTuple
from sklearn.metrics.scorer import check_scoring
from sklearn.utils.validation import _num_samples, indexable

class GridSearchCV(BaseSearchCV):
    """Exhaustive search over specified parameter values for an estimator, using Spark to
    distribute the computations.
    Important members are fit, predict.
    GridSearchCV implements a "fit" method and a "predict" method like
    any classifier except that the parameters of the classifier
    used to predict is optimized by cross-validation.
    Parameters
    ----------
    sc: the spark context
    estimator : object type that implements the "fit" and "predict" methods
        A object of that type is instantiated for each grid point.
    param_grid : dict or list of dictionaries
        Dictionary with parameters names (string) as keys and lists of
        parameter settings to try as values, or a list of such
        dictionaries, in which case the grids spanned by each dictionary
        in the list are explored. This enables searching over any sequence
        of parameter settings.
    scoring : string, callable or None, optional, default: None
        A string (see model evaluation documentation) or
        a scorer callable object / function with signature
        ``scorer(estimator, X, y)``.
    fit_params : dict, optional
        Parameters to pass to the fit method.
    n_jobs : int, default 1
        This parameter is not used and kept for compatibility.
    pre_dispatch : int, or string, optional
        This parameter is not used and kept for compatibility.
    iid : boolean, default=True
        If True, the data is assumed to be identically distributed across
        the folds, and the loss minimized is the total loss per sample,
        and not the mean loss across the folds.
    cv : integer or cross-validation generator, default=3
        A cross-validation generator to use. If int, determines
        the number of folds in StratifiedKFold if estimator is a classifier
        and the target y is binary or multiclass, or the number
        of folds in KFold otherwise.
        Specific cross-validation objects can be passed, see
        sklearn.cross_validation module for the list of possible objects.
    refit : boolean, default=True
        Refit the best estimator with the entire dataset.
        If "False", it is impossible to make predictions using
        this GridSearchCV instance after fitting.
        The refitting step, if any, happens on the local machine.
    verbose : integer
        Controls the verbosity: the higher, the more messages.
    error_score : 'raise' (default) or numeric
        Value to assign to the score if an error occurs in estimator fitting.
        If set to 'raise', the error is raised. If a numeric value is given,
        FitFailedWarning is raised. This parameter does not affect the refit
        step, which will always raise the error.
    Examples
    --------
    >>> from sklearn import svm, datasets
    >>> from spark_sklearn import GridSearchCV
    >>> from pyspark.sql import SparkSession
    >>> from spark_sklearn.util import createLocalSparkSession
    >>> spark = createLocalSparkSession()
    >>> iris = datasets.load_iris()
    >>> parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
    >>> svr = svm.SVC()
    >>> clf = GridSearchCV(spark.sparkContext, svr, parameters)
    >>> clf.fit(iris.data, iris.target)
    ...                             # doctest: +NORMALIZE_WHITESPACE +ELLIPSIS
    GridSearchCV(cv=None, error_score=...,
           estimator=SVC(C=1.0, cache_size=..., class_weight=..., coef0=...,
                         decision_function_shape=None, degree=..., gamma=...,
                         kernel='rbf', max_iter=-1, probability=False,
                         random_state=None, shrinking=True, tol=...,
                         verbose=False),
           fit_params={}, iid=..., n_jobs=1,
           param_grid=..., pre_dispatch=..., refit=...,
           scoring=..., verbose=...)
    >>> spark.stop(); SparkSession._instantiatedContext = None
    Attributes
    ----------
    grid_scores_ : list of named tuples
        Contains scores for all parameter combinations in param_grid.
        Each entry corresponds to one parameter setting.
        Each named tuple has the attributes:
            * ``parameters``, a dict of parameter settings
            * ``mean_validation_score``, the mean score over the
              cross-validation folds
            * ``cv_validation_scores``, the list of scores for each fold
    best_estimator_ : estimator
        Estimator that was chosen by the search, i.e. estimator
        which gave highest score (or smallest loss if specified)
        on the left out data. Not available if refit=False.
    best_score_ : float
        Score of best_estimator on the left out data.
    best_params_ : dict
        Parameter setting that gave the best results on the hold out data.
    scorer_ : function
        Scorer function used on the held out data to choose the best
        parameters for the model.
    Notes
    ------
    The parameters selected are those that maximize the score of the left out
    data, unless an explicit score is passed in which case it is used instead.
    The parameters n_jobs and pre_dispatch are accepted but not used.
    See Also
    ---------
    :class:`ParameterGrid`:
        generates all the combinations of a an hyperparameter grid.
    :func:`sklearn.cross_validation.train_test_split`:
        utility function to split the data into a development set usable
        for fitting a GridSearchCV instance and an evaluation set for
        its final evaluation.
    :func:`sklearn.metrics.make_scorer`:
        Make a scorer from a performance metric or loss function.
    """

    def __init__(self, estimator, param_grid, scoring=None, fit_params=None,
                 n_jobs=1, iid=True, refit=True, cv=None, verbose=0,
                 pre_dispatch='2*n_jobs', error_score='raise'):
        super(GridSearchCV, self).__init__(
            estimator, scoring, fit_params, n_jobs, iid,
            refit, cv, verbose, pre_dispatch, error_score)
        self.param_grid = param_grid
        self.grid_scores_ = None
        _check_param_grid(param_grid)

    def fit(self, X, y=None):
        """Run fit with all sets of parameters.
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Training vector, where n_samples is the number of samples and
            n_features is the number of features.
        y : array-like, shape = [n_samples] or [n_samples, n_output], optional
            Target relative to X for classification or regression;
            None for unsupervised learning.
        """
        return self._fit(X, y, ParameterGrid(self.param_grid))

    def _fit(self, X, y, parameter_iterable):
        """Actual fitting,  performing the search over parameters."""

        estimator = self.estimator
        cv = self.cv
        self.scorer_ = check_scoring(self.estimator, scoring=self.scoring)

        n_samples = _num_samples(X)
        X, y = indexable(X, y)

        if y is not None:
            if len(y) != n_samples:
                raise ValueError('Target variable (y) has a different number '
                                 'of samples (%i) than data (X: %i samples)'
                                 % (len(y), n_samples))
        cv = check_cv(cv, X, y, classifier=is_classifier(estimator))

        if self.verbose > 0:
            if isinstance(parameter_iterable, Sized):
                n_candidates = len(parameter_iterable)
                print("Fitting {0} folds for each of {1} candidates, totalling"
                      " {2} fits".format(len(cv), n_candidates,
                                         n_candidates * len(cv)))

        base_estimator = clone(self.estimator)

        param_grid = [(parameters, train, test)
                      for parameters in parameter_iterable
                      for (train, test) in cv]
        # Because the original python code expects a certain order for the elements, we need to
        # respect it.
        indexed_param_grid = list(zip(range(len(param_grid)), param_grid))
        par_param_grid = sc.parallelize(indexed_param_grid, len(indexed_param_grid))
        X_bc = sc.broadcast(X)
        y_bc = sc.broadcast(y)

        scorer = self.scorer_
        verbose = self.verbose
        fit_params = self.fit_params
        error_score = self.error_score
        fas = _fit_and_score

        def fun(tup):
            (index, (parameters, train, test)) = tup
            local_estimator = clone(base_estimator)
            local_X = X_bc.value
            local_y = y_bc.value
            res = fas(local_estimator, local_X, local_y, scorer, train, test, verbose,
                                  parameters, fit_params,
                                  return_parameters=True, error_score=error_score)
            return (index, res)
        indexed_out0 = dict(par_param_grid.map(fun).collect())
        out = [indexed_out0[idx] for idx in range(len(param_grid))]

        X_bc.unpersist()
        y_bc.unpersist()

        # Out is a list of triplet: score, estimator, n_test_samples
        n_fits = len(out)
        n_folds = len(cv)

        scores = list()
        grid_scores = list()
        for grid_start in range(0, n_fits, n_folds):
            n_test_samples = 0
            score = 0
            all_scores = []
            for this_score, this_n_test_samples, _, parameters in \
                    out[grid_start:grid_start + n_folds]:
                all_scores.append(this_score)
                if self.iid:
                    this_score *= this_n_test_samples
                    n_test_samples += this_n_test_samples
                score += this_score
            if self.iid:
                score /= float(n_test_samples)
            else:
                score /= float(n_folds)
            scores.append((score, parameters))
            # TODO: shall we also store the test_fold_sizes?
            grid_scores.append(_CVScoreTuple(
                parameters,
                score,
                np.array(all_scores)))
        # Store the computed scores
        self.grid_scores_ = grid_scores

        # Find the best parameters by comparing on the mean validation score:
        # note that `sorted` is deterministic in the way it breaks ties
        best = sorted(grid_scores, key=lambda x: x.mean_validation_score,
                      reverse=True)[0]
        self.best_params_ = best.parameters
        self.best_score_ = best.mean_validation_score

        if self.refit:
            # fit the best estimator using the entire dataset
            # clone first to work around broken estimators
            best_estimator = clone(base_estimator).set_params(
                **best.parameters)
            if y is not None:
                best_estimator.fit(X, y, **self.fit_params)
            else:
                best_estimator.fit(X, **self.fit_params)
            self.best_estimator_ = best_estimator
        return self

In [4]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cross_validation import cross_val_score
from sklearn.feature_selection import SelectKBest, chi2

class NotSparseTfidfTransformer(TfidfTransformer):
    # XGBoost and preprocessing/standard scaler don't
    # work with sparse data
    # def fit_transform(self, X, Y=None):
    #     r = super(NotSparseTfidfTransformer, self).fit_transform(X)
    #     return r.toarray()

    def transform(self, X):
        r = super(NotSparseTfidfTransformer, self).transform(X)
        return r.toarray()

def run(analysis, shuffle=True):
    l = Load(analysis=analysis, min_num_occurence_of_category=10)
    cv_param = StratifiedKFold(l.Y, n_folds=3, shuffle=shuffle)
    pipeline = Pipeline([
        ('vect', CountVectorizer(min_df=1,
                                 # token_pattern=r'\b\w+\b'
                                 token_pattern=r'\b[a-zA-Z_]+\b')),
        ('tfidf', NotSparseTfidfTransformer()),
        ('chi2', SelectKBest(chi2)),
        ('scaler', preprocessing.StandardScaler()),
        # ('clf', SGDClassifier(loss='hinge', penalty='l2',
        #                       n_iter=100000,
        #                       random_state=random.randint(0, 1000))),
        ('clf', RandomForestClassifier())
        # ('clf', XGBClassifier())
    ])
    param_grid = {'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
                  'vect__max_df': (0.5, 0.75, 1.0),
                  'tfidf__use_idf': (True, False),
                  'tfidf__norm': ('l1', 'l2'),
                  'chi2__k': (100, 200, 500),
                  #
                  'scaler__with_mean': (True, False),
                  'clf__n_estimators': (100, 200, 500,)}
    #param_grid = {'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    #              'vect__max_df': (0.5,),
    #              'tfidf__use_idf': (True,),
    #              'tfidf__norm': ('l1',),
                  # 'chi2__k': (100, 200, 500),
                  #
    #              'scaler__with_mean': (True,),
    #              'clf__n_estimators': (100,)}
    clf = GridSearchCV(pipeline,
                       param_grid=param_grid,
                       cv=3,
                       n_jobs=-1)
    res = cross_val_score(clf,
                          l.X_text,
                          cv=cv_param,
                          y=l.Y)
    return sum(res) / len(res)


In [ ]:
def run_n(analysis, shuffle, n):
    return [run(analysis, shuffle=shuffle) for _ in range(n)]
    

In [ ]:
res = {}
categories = [Analysis.VERIFIED_CATEGORY, Analysis.VERIFIED_RELIABLE, Analysis.GUESSED_RELIABLE]
to_browse = sum([[(x, False), (x, True)] for x in categories], [])
for analysis, shuffle in to_browse:
    res["cat={}, shuffle={}".format(analysis, shuffle)] = run_n(analysis, shuffle, 30)
print(res)

In [ ]:
# sc.stop()

In [ ]:
import json

def dump(res):
    buggy_res_path = '/opt/shared_venvs/results/buggy_res.json'
    with open(buggy_res_path, 'w') as f:
        json.dump(res, f)

In [ ]:
# dump(res)